In [7]:
import os
import zipfile

local_zip="./tmp/horse-or-human.zip"
zip_ref=zipfile.ZipFile(local_zip,"r")
zip_ref.extractall("./tmp/horse-or-human")
zip_ref.close()

In [10]:
train_horse_dir=os.path.join('./tmp/horse-or-human/horses')
train_human_dir=os.path.join('./tmp/horse-or-human/humans')

In [11]:
train_horse_names=os.listdir(train_horse_dir)
print(train_horse_names[:10])
train_human_names=os.listdir(train_human_dir)
print(train_human_names[:10])

['horse01-0.png', 'horse01-1.png', 'horse01-2.png', 'horse01-3.png', 'horse01-4.png', 'horse01-5.png', 'horse01-6.png', 'horse01-7.png', 'horse01-8.png', 'horse01-9.png']
['human01-00.png', 'human01-01.png', 'human01-02.png', 'human01-03.png', 'human01-04.png', 'human01-05.png', 'human01-06.png', 'human01-07.png', 'human01-08.png', 'human01-09.png']


In [12]:
print("total horse images : ",len(train_horse_names))
print("total_human_images : ",len(train_human_names))

total horse images :  500
total_human_images :  527


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

nrows=4
ncols=4
pic_index=0

fig=plt.gcf()
fig.set_size_inches(ncols*4,nrows*4)
pic_index+=8


In [13]:
import tensorflow as tf
from tensorflow import keras

In [18]:
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Flatten,Dense

In [19]:
model=keras.models.Sequential([
    Conv2D(16,(3,3),activation='relu',input_shape=(300,300,3)),
    MaxPooling2D(2,2),
    Conv2D(32,(3,3),activation="relu"),
    MaxPooling2D(2,2),
    Conv2D(64,(3,3),activation="relu"),
    MaxPooling2D(2,2),
    Conv2D(64,(3,3),activation="relu"),
    MaxPooling2D(2,2),
    Conv2D(64,(3,3),activation="relu"),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(512,activation="relu"),
    Dense(1,activation="sigmoid")
    
])

In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 298, 298, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 149, 149, 16)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 147, 147, 32)      4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 73, 73, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 71, 71, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 35, 35, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 33, 33, 64)        3

In [24]:
from tensorflow.keras.optimizers import RMSprop

model.compile(loss="binary_crossentropy",
             optimizer=RMSprop(lr=0.001),
             metrics=['accuracy'])

In [29]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen=ImageDataGenerator(rescale=1./255)
validation_datagen=ImageDataGenerator(rescale=1./255)

train_generator=train_datagen.flow_from_directory(
    './tmp/horse-or-human/',
    target_size=(300,300),
    batch_size=128,
    class_mode="binary")

validation_generator = validation_datagen.flow_from_directory(
        './tmp/validation-horse-or-human/',  
        target_size=(300, 300),
        batch_size=32,
        class_mode='binary')

Found 1027 images belonging to 2 classes.
Found 256 images belonging to 2 classes.


In [30]:
history=model.fit(
train_generator,
steps_per_epoch=8,
epochs=15,
verbose=1,
validation_data=validation_generator,
validation_steps=8)

Train for 8 steps, validate for 8 steps
Epoch 1/15


W0615 16:21:37.245184 15584 deprecation.py:323] From c:\users\home\anaconda3\envs\tensorflow\lib\site-packages\tensorflow_core\python\ops\nn_impl.py:183: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


8/8 [==============================] - 131s 16s/step - loss: 0.9059 - accuracy: 0.5068 - val_loss: 0.6698 - val_accuracy: 0.5000
Epoch 2/15
8/8 [==============================] - 153s 19s/step - loss: 0.7896 - accuracy: 0.6741 - val_loss: 0.3816 - val_accuracy: 0.8398
Epoch 3/15
8/8 [==============================] - 125s 16s/step - loss: 0.5666 - accuracy: 0.7953 - val_loss: 0.3258 - val_accuracy: 0.8672
Epoch 4/15
8/8 [==============================] - 67s 8s/step - loss: 0.3414 - accuracy: 0.8420 - val_loss: 0.8608 - val_accuracy: 0.7734
Epoch 5/15
8/8 [==============================] - 69s 9s/step - loss: 0.2102 - accuracy: 0.9043 - val_loss: 0.7888 - val_accuracy: 0.7344
Epoch 6/15
8/8 [==============================] - 80s 10s/step - loss: 0.1573 - accuracy: 0.9377 - val_loss: 1.3791 - val_accuracy: 0.8008
Epoch 7/15
8/8 [==============================] - 81s 10s/step - loss: 0.4722 - accuracy: 0.8209 - val_loss: 1.3758 - val_accuracy: 0.7891
Epoch 8/15
8/8 [=====================

In [ ]:
import numpy as np
from keras.preprocessing import image

file_path=""
img=image.load_img(file_path,target_size=(300,300))
x=image.img_to_array(img)
x=np.expand_dims(x,axis=0)

images=np.vstack([x])
classes=model.predict(images,batch_size=10)
